In [8]:
import numpy as np
import math as math
import pandas as pd
import datetime as date
import pandas_datareader as web
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
import mysql.connector
from datetime import timedelta, datetime

commercial_banks = ['ADBL','BOKL','CCBL']

sectorss = { 'corporate_debentures':['NICAD8283', 'NBLD85'],
            'microfinance':['ACLBSL','ALBSL','CBBL','CLBSL','DDBL','FMDBL','FOWAD','GMFBS','GILB','GBLBS','GLBSL','ILBS','JALPA','JSLBB','JBLB','KMCDB','KLBSL','LLBS','MLBSL','MSLB','MKLB','MLBS','MERO','MMFDB','MLBBL','NSLB','NLBBL','NESDO','NICLBSL','NUBL','RULB','RMDC','RSDC','SABSL','SDLBSL','SMATA','SLBSL','SKBBL','SMFDB','SMB','SWBBL','SMFBS','SLBBL','USLB','VLBS','WNLB'],
            'commercial_banks':['ADBL','BOKL','CCBL','CZBIL','CBL','EBL','GBIME','KBL','LBL','MBL','MEGA','NABIL','NBL','NCCB','SBI','NICA','NMB','PRVU','PCBL','SANIMA','SBL','SCB','SRBL'],
            'non_life_insurance':['AIL','EIC','GIC','HGI','IGI','LGIL','NIL','NICL','NLG','PRIN','PIC','PICL','RBCL','SIC','SGI','SICL','SIL','UIC'],
            'hydro_powers':['AKJCL','API','AKPL','AHPC','BARUN','BNHC','BPCL','CHL','CHCL','DHPL','GHL','GLH','HDHPC','HURJA','HPPL','JOSHI','KPCL','KKHC','LEC','MBJC','MKJC','MEN','MHNL','NHPC','NHDL','NGPL','NYADI','PMHPL','PPCL','RADHI','RHPL','RURU','SAHAS','SPC','SHPC','SJCL','SSHL','SHEL','SPDL','TPC','UNHPL','UMRH','UMHL','UPCL','UPPER'],
            'life_insurance':['ALICL','GLICL','JLI','LICN','NLICL','NLIC','PLI','PLIC','RLI','SLI','SLICL','ULI'],
            'finance':['BFC','CFCL','GFCL','GMFIL','GUFL','ICFC','JFL','MFIL','MPFL','NFS','PFL','PROFL','RLFL','SFCL','SIFC'],
            'tradings':['BBC', 'STC'],
            'manufacturing_and_processing':['BNT', 'HDL', 'SHIVM', 'UNL'],
            'investment':['CHDC', 'CIT', 'ENL', 'HIDCL', 'NIFRA', 'NRN'],
            'hotels':['CGH', 'OHL', 'SHL', 'TRH'],
            'development_banks':['CORBL','EDBL','GBBL','GRDBL','JBBL','KSBBL','KRBL','LBBL','MLBL','MDB','MNBBL','NABBC','SAPDBL','SADBL','SHINE','SINDU'],
            'mutual_fund':['KEF','LUK','NEF','NIBLPF'],
            'other':['NTC', 'NRIC']
           }

db = mysql.connector.connect(host='localhost', user='root', database='stock')
cursor = db.cursor(buffered=True)

for bank_name in commercial_banks:
    sql = f"select * from commercial_banks where Scrip = '{bank_name}'"
    cursor.execute(sql)
    result = cursor.fetchall()
    
    df = pd.DataFrame(data = result, columns=['Scrip','Time','Close','Open','High','Low','Volume'])
    
    df1 = df[['Time']].astype('datetime64[s]')
    
    df['Time']=df1['Time']
    
    df2 = df.set_index('Time')
    
    df3=df2[['Close']]
    
    data = df3.filter(['Close'])
    dataset = data.values
    training_data_len = math.ceil( len(dataset) * .7 )
    
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    
    train_data = scaled_data[0:training_data_len , :]
    x_train = []
    y_train = []
    for i in range(70, len(train_data)):
        x_train.append(train_data[i-70:i, 0])
        y_train.append(train_data[i, 0])
        if i<=71:
            print(x_train)
            print(y_train)
            print()
    x_train, y_train = np.array(x_train), np.array(y_train)
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dense(25))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    model.fit(x_train, y_train, batch_size=1, epochs=1)
    
    test_data = scaled_data[training_data_len - 70: , :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(70, len(test_data)):
        x_test.append(test_data[i-70:i, 0])
        
    x_test = np.array(x_test)
    
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    
    new_df = df3.filter(['Close'])
    last_70_days = new_df[-70:].values
    last_70_days_scaled = scaler.transform(last_70_days)
    X_test = []
    X_test.append(last_70_days_scaled)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    pred_price = model.predict(X_test)
    pred_price = scaler.inverse_transform(pred_price)
    
    
    int_price = pred_price[0][0]
    int_price1 = format(int_price, ".2f")
    
    price = float(int_price1)
    
    today_price = df3.tail(1)
    
    db_predict = mysql.connector.connect(host='localhost', user='root', password='',
                                         database='stock')
    
    today = datetime.now() # get date and time today
    delta = timedelta(days=1) #initialize delta
    date = today + delta # add the delta days
    tomorrow_date = date.strftime('%Y-%m-%d') # format it
    
    
    cursor_predict = db_predict.cursor()
    Date = tomorrow_date ### Date of prediction
    Predicted_value = price ### predicted price of next day
    data = (bank_name, Date, Predicted_value)
    sql = "INSERT INTO prediction VALUES (%s,%s,%s)" ##prediction is table name where data is to be inserted
    cursor_predict.execute(sql, data)
    db_predict.commit()

[array([0.01732926, 0.01427115, 0.01325178, 0.01223242, 0.01325178,
       0.01019368, 0.01121305, 0.01121305, 0.01121305, 0.01019368,
       0.01223242, 0.01630989, 0.01630989, 0.01121305, 0.01325178,
       0.01121305, 0.01121305, 0.00917431, 0.01019368, 0.01019368,
       0.00917431, 0.01019368, 0.00917431, 0.00917431, 0.00815494,
       0.00815494, 0.00713558, 0.00815494, 0.01019368, 0.00713558,
       0.00815494, 0.00815494, 0.00611621, 0.00713558, 0.00815494,
       0.00713558, 0.00713558, 0.00815494, 0.00611621, 0.00407747,
       0.00509684, 0.00509684, 0.0030581 , 0.00203874, 0.00203874,
       0.00203874, 0.00101937, 0.        , 0.0030581 , 0.00509684,
       0.00611621, 0.00203874, 0.00509684, 0.00407747, 0.00509684,
       0.00407747, 0.00509684, 0.00407747, 0.00509684, 0.00509684,
       0.00509684, 0.00509684, 0.00611621, 0.01325178, 0.0224261 ,
       0.03465851, 0.02956167, 0.02344546, 0.02140673, 0.0224261 ])]
[0.02854230377166156]

[array([0.01732926, 0.01427115, 0.01

ProgrammingError: 1146 (42S02): Table 'stock.prediction' doesn't exist